In [1]:
!pwd

/home/rsebastian/code/elixier-scraper/notebook


In [7]:
import pandas as pd
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import numpy as np
import pickle

In [32]:
oer = pd.read_csv('../data/elixier_physics_all.csv', index_col=0)

In [33]:
bildungsebene = [be.lower() for be in [
    "Sekundarstufe I", # (1645)
    "Sekundarstufe II", # (1118)
    "Primarstufe", # (426)
    "Hochschule", # (85)
    "Berufliche Bildung", # (68)
    "Elementarbildung", # (28)
    "Fort- und Weiterbildung" # (27)
]]

In [34]:
bildungsebene_clean = []
for val in oer.bildungsebene:
    if type(val) == str:
        str_out = []
        val_lower = val.lower()
        for be in bildungsebene:
            if be in val_lower:
                str_out.append(be)
        bildungsebene_clean.append(str_out)
    else:
        bildungsebene_clean.append([])
oer['bildungsebene_clean'] = bildungsebene_clean

In [35]:
oer.bildungsebene_clean.apply(lambda x: ' '.join(x) if type(x) == list else '').value_counts()

sekundarstufe i sekundarstufe ii                                                          962
                                                                                          818
sekundarstufe i                                                                           584
sekundarstufe i primarstufe                                                               268
primarstufe                                                                                70
sekundarstufe i sekundarstufe ii primarstufe                                               58
sekundarstufe i sekundarstufe ii hochschule                                                38
sekundarstufe i berufliche bildung                                                         24
sekundarstufe i sekundarstufe ii hochschule berufliche bildung fort- und weiterbildung     20
sekundarstufe i primarstufe elementarbildung                                               16
sekundarstufe i sekundarstufe ii berufliche bildung         

In [8]:
def get_ignore(url):
    try:
        return requests.get(url)
    except Exception as e:
        return e

In [4]:
responses = [get_ignore(u) for u in tqdm(oer.url)]

  0%|                                                                                                               | 0/2899 [00:00<?, ?it/s]/home/rsebastian/.anaconda3/envs/elixier/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2899/2899 [59:07<00:00,  1.22s/it]


In [8]:
# with open('../data/responses', 'wb') as f:
#     pickle.dump(responses, f)
with open('../data/responses', 'rb') as f:
    responses = pickle.load(f)

In [17]:
success_mask = pd.Series(responses).apply(lambda r: (not isinstance(r, Exception)) and (r.status_code == 200))

In [31]:
oer = oer[success_mask]

TypeError: tuple indices must be integers or slices, not Series

In [30]:
oer.columns

AttributeError: 'tuple' object has no attribute 'columns'

In [26]:
descriptions = oer['lange beschreibung']
existing_descriptions = descriptions[descriptions.notna()]
existing_descriptions.shape[0]

365

In [12]:
rs = rs[success_mask]

In [13]:
content_type = rs.apply(lambda r: r.headers['Content-Type'].lower().split(';')[0])
content_type.value_counts().sort_values(ascending=False)

text/html                                                                  2559
application/pdf                                                             160
audio/mpeg                                                                   29
application/vnd.openxmlformats-officedocument.wordprocessingml.document      14
application/zip                                                              13
application/octet-stream                                                      3
application/x-shockwave-flash                                                 2
image/jpeg                                                                    2
application/x-msdos-program                                                   1
image/gif                                                                     1
dtype: int64

In [20]:
html_responses = rs[content_type.str.contains('text/html')]

In [45]:
def title(soup):
    maybe_title = soup.find('title')
    if maybe_title is None:
        return ""
    else:
        return maybe_title.text

In [36]:
def content_paras(soup):
    return [p.text for p in soup.find_all('p') if len(p.text.strip()) > 200]

In [37]:
def multidict(kvs):
    out = dict()
    for k, v in kvs:
        if k in out:
            out[k].append(v)
        else:
            out[k] = [v]
    return out

def headings(soup):
    return multidict([(t.name, t.text.strip()) for i in range(6) for t in soup.find_all(f'h{i}')])

In [70]:
def content(response):
    url = response.url
    domain = url.split('/')[2]
    soup = BeautifulSoup(response.content)
    return {'paras': content_paras(soup), 
            'headings': headings(soup), 
            'title': title(soup), 
            'url': url,
            'domain': domain}

In [53]:
def content_counts(cont):
    out = {
        'num_paras': len(cont['paras']),
        'tatal_para_words': sum([len(p.split(' ')) for p in cont['paras']]),
        'title_size': len(cont['title']),
        'url': cont['url']
    }
    for h_type in cont['headings']:
        out[h_type + '_size'] = len(cont['headings'][h_type])
    return out

In [72]:
contents = html_responses.apply(content)

In [73]:
with open('../data/cache/parsed_contents.pkl', 'wb') as f:
    pickle.dump(contents, f)

In [58]:
content_estimates = pd.DataFrame([content_counts(c) for c in contents])

In [67]:

content_estimates.groupby('url').aggregate({
    'num_paras': ['min', 'max', 'mean'],
    'avg_para_size_words': ['min', 'max', 'mean'],
    'h1_size': ['min', 'mean'],
    'h2_size': ['min', 'mean'],
    'h3_size': ['min', 'mean'],
    'h4_size': ['min', 'mean'],
    'h5_size': ['min', 'mean'],
    'url': ['count']
}).sort_values(by=('url', 'count'), ascending=False).iloc[:50]

num_paras                  \
                                             min  max       mean   
url                                                                
www.leifiphysik.de                             0   46   6.048662   
www.lehrer-online.de                           0   25   3.968153   
www.supra-lernplattform.de                     0   25   4.402174   
www.bildung-lsa.de                             0    1   0.825000   
www.planet-schule.de                           0   15   2.824324   
de.wikipedia.org                               0  228  30.145455   
www.abi-physik.de                              0   11   4.226415   
www.zum.de                                     0    2   0.090909   
www.heise.de                                   3   31  12.807692   
www.komm-mach-mint.de                          1    5   2.038462   
digitallearninglab.de                          0    3   0.142857   
www.dlr.de                                     1   14   4.400000   
primas.ph-freiburg.de                          1   15   5.235294   
www.science-on-stage.de                        2    5   2.562500   
www.zauberhafte-physik.net                    13   24  13.875000   
www.esa.int                                    0   10   0.875000   
www.wissenschaftsjahr.de                       2   17   6.266667   
blogs.helmholtz.de                             4    8   5.230769   
lernarchiv.bildung.hessen.de                   0    9   0.769231   
physikforkids.de                               0    2   0.916667   
www.bildungsserver.de                          1   16   6.250000   
virtuelle-experimente.de                       0    2   0.416667   
unterrichten.zum.de                            1    1   1.000000   
www.wissenschaft-schulen.de                    0    3   0.500000   
www.tempolimit-lichtgeschwindigkeit.de         1    9   3.555556   
physikaufgaben.de                              2    2   2.000000   
jugend-und-bildung.de                          3    5   4.000000   
mint-zirkel.de                                 0   13   4.750000   
roentgenmuseum.de                              8   14   8.750000   
www.walter-fendt.de                            0   27   3.875000   
www.meine-forscherwelt.de                      0    0   0.000000   
platform.govie.de                              0    0   0.000000   
esero.de                                       3    3   3.000000   
physik-am-auto.de                              2   11   6.400000   
www.schulentwicklung.nrw.de                    1   14   4.000000   
www.dguv-lug.de                                2    4   3.200000   
medienportal.siemens-stiftung.org              2   15   4.600000   
www.unterrichtsmaterial-schule.de              0    3   1.400000   
physik.uibk.ac.at                              0    0   0.000000   
www.fis.uni-bonn.de                            1    1   1.000000   
physikunterricht-online.de                     3    8   6.500000   
idw-online.de                                  1    3   1.750000   
www.kindernetz.de                              0    0   0.000000   
www.youtube.com                                0    0   0.000000   
www.compass-project.eu                         0    0   0.000000   
www.seilnacht.tuttlingen.com                   0    0   0.000000   
www.sachsen.schule                             0    2   0.666667   
www.schule-bw.de                               1   12   5.333333   
steinphysik.de                                 1    5   3.666667   
space2school.de                                0    2   1.333333   

                                       avg_para_size_words               \
                                                       min          max   
url                                                                       
www.leifiphysik.de                                0.000000   370.000000   
www.lehrer-online.de                              0.000000   357.812500   
www.supra-lernplattform.de                        0.000000 